2150 edges, 546 nodes, 9014 edges in the line graph

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00


In [ ]:
import autoencoder
from autoencoder import VGAE, GAE, EdgeDecoder, DirectedEdgeDecoder, InnerProductDecoder, DirectedInnerProductDecoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
import copy
import torch
from torch_geometric.utils.convert import from_networkx
from torch_geometric.utils import train_test_split_edges
from scipy.stats import ranksums
from torch_geometric.nn import GraphConv, SAGEConv, GATConv, GCNConv
import torch.nn.functional as F
from torch_geometric.transforms import LineGraph, RandomNodeSplit, RandomLinkSplit
from torch_geometric import seed_everything

seed_everything(42)

In [ ]:
node_rename = {node: id for id, node in enumerate(range(388, 934))}
nodefile = 'ChicagoSketch_node.tntp'
node = pd.read_csv(nodefile, sep='\t', usecols=['node', 'X', 'Y'])
flowfile = 'ChicagoSketch_flow.tntp'
colname = 'Volume '
flow = pd.read_csv(flowfile, sep='\t', usecols=['From ', 'To ', colname])

node['node'] = node['node'].map(node_rename)
node = node[node['node'].notna()]

flow['From '] = flow['From '].map(node_rename)
flow['To '] = flow['To '].map(node_rename)
flow = flow[(flow['From '].notna()) & (flow['To '].notna())]
flow.drop(flow[flow[colname] <= 0].index, inplace=True)
flow[colname] = np.log(flow[colname])

scaler = StandardScaler()
node[['X', 'Y']] = scaler.fit_transform(node[['X', 'Y']].values)
# minmax = MinMaxScaler()
# flow[[colname]] = minmax.fit_transform(flow[[colname]].values)

df = flow.rename(columns={'From ': 's', 'To ': 'r', colname: 'w'})
df1 = pd.merge(df, node, how='left', left_on='s', right_on='node')[['s', 'r', 'w', 'X', 'Y']].rename(columns={'X': 'X1', 'Y': 'Y1'})
df2 = pd.merge(df1, node, how='left', left_on='r', right_on='node')[['s', 'r', 'w', 'X1', 'Y1', 'X', 'Y']].rename(columns={'X': 'X2', 'Y': 'Y2'})
df2['feat'] = df2[['X1', 'Y1', 'X2', 'Y2']].values.tolist()

edge_name_to_y = {(s, r): w for s, r, w in df2[['s', 'r', 'w']].values}
edge_name_to_x = {(s, r): feat for s, r, feat in df2[['s', 'r', 'feat']].values}

In [ ]:
import geopandas as gpd
nodefile = 'anaheim_nodes.geojson'
nodes = gpd.read_file(nodefile)
nodes_df = pd.DataFrame(nodes)
nodes_df[['X', 'Y']] = nodes_df['geometry'].astype(str).str.split('(').str[-1].str.split(')').str[0].str.split(' ', expand=True).astype(np.float32)
node = nodes_df.rename(columns={'id': 'node'})

node_rename = {node: id for id, node in enumerate(range(1, 417))}
node['node'] = node['node'].map(node_rename)

flowfile = 'Anaheim_flow.tntp'
colname = 'Volume '
flow = pd.read_csv(flowfile, sep='\t', usecols=['From ', 'To ', colname])

flow['From '] = flow['From '].map(node_rename)
flow['To '] = flow['To '].map(node_rename)
flow = flow[(flow['From '].notna()) & (flow['To '].notna())]
flow.drop(flow[flow[colname] <= 0].index, inplace=True)
flow[colname] = np.log(flow[colname])

scaler = StandardScaler()
node[['X', 'Y']] = scaler.fit_transform(node[['X', 'Y']].values)
# minmax = MinMaxScaler()
# flow[[colname]] = minmax.fit_transform(flow[[colname]].values)

df = flow.rename(columns={'From ': 's', 'To ': 'r', colname: 'w'})
df1 = pd.merge(df, node, how='left', left_on='s', right_on='node')[['s', 'r', 'w', 'X', 'Y']].rename(columns={'X': 'X1', 'Y': 'Y1'})
df2 = pd.merge(df1, node, how='left', left_on='r', right_on='node')[['s', 'r', 'w', 'X1', 'Y1', 'X', 'Y']].rename(columns={'X': 'X2', 'Y': 'Y2'})
df2['feat'] = df2[['X1', 'Y1', 'X2', 'Y2']].values.tolist()

edge_name_to_y = {(s, r): w for s, r, w in df2[['s', 'r', 'w']].values}
edge_name_to_x = {(s, r): feat for s, r, feat in df2[['s', 'r', 'feat']].values}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
G = nx.from_pandas_edgelist(df2, source='s', target='r', edge_attr='w', create_using=nx.DiGraph())
airport = from_networkx(G)
airport.x = torch.from_numpy(node[['X', 'Y']].values).to(torch.float32)
print(airport)

G_line_graph = nx.line_graph(G, create_using=nx.DiGraph())
airport_line_graph = from_networkx(G_line_graph)
airport_line_graph.x = torch.from_numpy(np.vstack([edge_name_to_x[e] for e in G_line_graph.nodes])).to(torch.float32)
airport_line_graph.y = torch.from_numpy(np.vstack([edge_name_to_y[e] for e in G_line_graph.nodes])).to(torch.float32)
print(airport_line_graph)

split = RandomNodeSplit(num_val=0.1, num_test=0.4)
data = split(airport_line_graph)
data = data.to(device)

Data(edge_index=[2, 858], w=[858], num_nodes=413, x=[416, 2])
Data(edge_index=[2, 2214], num_nodes=858, x=[858, 4], y=[858, 1])


In [ ]:
edge_array = np.array(list(dict(G_line_graph.nodes).keys()))
edge_index_train = edge_array[data.train_mask.detach().numpy()]
edge_index_val = edge_array[data.val_mask.detach().numpy()]
edge_index_calib_test = edge_array[data.test_mask.detach().numpy()]

edge_weight_train = torch.Tensor(np.stack([edge_name_to_y[tuple(edge)] for edge in edge_index_train])).to(device)
edge_weight_val = torch.Tensor(np.stack([edge_name_to_y[tuple(edge)] for edge in edge_index_val])).to(device)
edge_weight_calib_test = torch.Tensor(np.stack([edge_name_to_y[tuple(edge)] for edge in edge_index_calib_test])).to(device)

edge_index_train = torch.LongTensor(edge_index_train).T.to(device)
edge_index_val = torch.LongTensor(edge_index_val).T.to(device)
edge_index_calib_test = torch.LongTensor(edge_index_calib_test).T.to(device)
edge_tensor = torch.LongTensor(edge_array).T.to(device)
edge_tensor2 = edge_tensor
edge_index_val2=edge_index_val
edge_index_train2=edge_index_train
edge_index_calib_test2=edge_index_calib_test

edge_weight_train2=edge_weight_train
edge_weight_val2=edge_weight_val
edge_weight_calib_test2=edge_weight_calib_test

In [ ]:
edge_weight_gae_training = [edge_name_to_y[tuple(edge)] if train else 1.0 for edge, train in zip(edge_array, data.train_mask.detach().numpy())]
edge_weight_gae_training = torch.Tensor(edge_weight_gae_training).to(device) # torch.ones(edge_array.shape[0]).to(device)
edge_weight_gae_training2 = edge_weight_gae_training

# w_min, w_max = edge_weight_gae_training.min(), edge_weight_gae_training.max()
# edge_weight_gae_training = (edge_weight_gae_training - w_min) / (w_max - w_min)# * 0.5 + 0.5

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, gconv=SAGEConv):
        super().__init__()
        self.conv1 = gconv(in_channels, hidden_channels)
        self.conv2 = gconv(hidden_channels, out_channels)
        self.gconv = gconv

    def forward(self, x, edge_index, edge_weight=None):
        if self.gconv not in [GCNConv, GraphConv]:
            edge_weight = None
        if edge_weight is not None:
            edge_weight = (edge_weight).sigmoid()
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [ ]:
class DirectedGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, gconv=SAGEConv):
        super().__init__()
        self.layers = [in_channels, hidden_channels, out_channels]
        self.num_layers = len(self.layers) - 1
        self.source = torch.nn.ModuleList()
        self.target = torch.nn.ModuleList()
        self.gconv = gconv
        for n_in, n_out in zip(self.layers[:-1], self.layers[1:]):
            self.source.append(gconv(n_in, n_out))
            self.target.append(gconv(n_in, n_out))

    def forward(self, s, t, edge_index, edge_weight=None):
        if self.gconv not in [GCNConv, GraphConv]:
            edge_weight = None
        if edge_weight is not None:
            edge_weight = (edge_weight).sigmoid()
        for layer_id, (layer_s, layer_t) in enumerate(zip(self.source, self.target)):
            s_new = layer_s(t, edge_index, edge_weight)
            t_new = layer_t(s, torch.flip(edge_index, [0]), edge_weight)
            if layer_id < self.num_layers - 1:
                s_new = s_new.relu()
                t_new = t_new.relu()
                s_new = F.dropout(s_new, p=0.5, training=self.training)
                t_new = F.dropout(t_new, p=0.5, training=self.training)
            s = s_new
            t = t_new

        return s, t

In [ ]:
def cqr2_new(cal_labels2, cal_labels, cal_lower, cal_upper, val_labels2, val_labels, val_lower, val_upper, n, alpha):
    cal_scores = np.maximum((cal_labels-cal_upper)/ np.abs(cal_labels2), (cal_lower-cal_labels) / np.abs(cal_labels2))
    qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, method='higher')
    prediction_sets = [val_lower - qhat * np.abs(val_labels2), val_upper + qhat*np.abs(val_labels2)]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    # eff = np.mean(val_upper + qhat - (val_lower - qhat))
    #print(2* qhat * np.abs(val_labels2))
    eff = np.mean(prediction_sets[1] - prediction_sets[0])
    return prediction_sets, cov, eff


In [ ]:
def cqr_new(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n, alpha):
    cal_scores = np.maximum(cal_labels-cal_upper, cal_lower-cal_labels)
    cal_scores = cal_scores / np.abs(cal_upper - cal_lower)
    qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, method='higher')
    prediction_sets = [val_lower - qhat * np.abs(val_upper - val_lower), val_upper + qhat * np.abs(val_upper - val_lower)]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    eff = np.mean(prediction_sets[1] - prediction_sets[0])
    return prediction_sets, cov, eff

def cqr(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n, alpha):
    cal_scores = np.maximum(cal_labels-cal_upper, cal_lower-cal_labels)
    qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, method='higher')
    prediction_sets = [val_lower - qhat, val_upper + qhat]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    eff = np.mean(val_upper + qhat - (val_lower - qhat))
    return prediction_sets, cov, eff

def qr(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n, alpha):
    prediction_sets = [val_lower, val_upper]
    cov = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
    eff = np.mean(val_upper - val_lower)
    return prediction_sets, cov, eff


In [ ]:
def worst_slice_coverage(x, edge_index_calib_test, idx, val_labels, prediction_sets):
    if torch.is_tensor(x):
        x = x.detach().numpy()
    if torch.is_tensor(edge_index_calib_test):
        edge_index_calib_test = edge_index_calib_test.detach().numpy()
    xtest = np.hstack([x[edge_index_calib_test[0, ~idx]], x[edge_index_calib_test[1, ~idx]]])
    ntest = xtest.shape[0]
    nfeat = xtest.shape[1]
    xtest_test = xtest[:ntest//4]
    unitvec = np.random.randn(nfeat, 1000)
    unitvec = unitvec / np.sqrt((unitvec**2).sum(axis=0))
    # ab_range = np.quantile((xtest_test @ unitvec).flatten(), np.linspace(0, 1, 11))
    values = (xtest_test @ unitvec).flatten()
    ab_range = np.linspace(values.min(), values.max(), 10)

    ws_cov_min = None
    for delta in np.linspace(0.1, 0.5, 5):
        ws_cov = 1
        ws_a = None
        ws_b = None
        ws_vec = None
        for vec in unitvec.T:
            value_vec = xtest_test @ vec.reshape(-1, 1)
            for a, b in zip(ab_range[:-1], ab_range[1:]):
                contained = np.bitwise_and(value_vec > a, value_vec < b).flatten()
                if contained.mean() > delta:
                    conditional_cov = ((val_labels[:ntest//4][contained] >= prediction_sets[0][:ntest//4][contained]) & (val_labels[:ntest//4][contained] <= prediction_sets[1][:ntest//4][contained])).mean()
                    if conditional_cov < ws_cov:
                        #print(f"Worst-Slice coverage = {conditional_cov:.4f}")
                        ws_cov = conditional_cov
                        ws_a = a
                        ws_b = b
                        ws_vec = vec
        if ws_vec is None:
            return None
        xtest_true = xtest[ntest//4:]
        value_vec = xtest_true @ ws_vec.reshape(-1, 1)
        contained = np.bitwise_and(value_vec > ws_a, value_vec < ws_b).flatten()
        ws_cov_true = ((val_labels[ntest//4:][contained] >= prediction_sets[0][ntest//4:][contained]) & (val_labels[ntest//4:][contained] <= prediction_sets[1][ntest//4:][contained])).mean()
        if ws_cov_min is not None and ws_cov_true < ws_cov_min:
            ws_cov_min = ws_cov_true
        elif ws_cov_min is None and ~np.isnan(ws_cov_true):
            ws_cov_min = ws_cov_true
    return ws_cov_min


In [ ]:
EPOCHS = 5001
ALPHA = 0.05
LR = 0.01
WD = 5e-4
HIDDEN = 8
OUT = 2

######################
SCORE = 'cqr2_new'
GNNCONV = GATConv
# CP_or_CQR
# Dataset
# DiGAE_or_GAE_or_LGNN

In [ ]:
def train_linegraph(alpha=ALPHA, val=False):
    if val:
        model.eval()
    else:
        model.train()
    mask = data.val_mask if val else data.train_mask
    out = model(data.x, data.edge_index)
    out1=out
    label = data.y[mask]
    mse_loss = F.mse_loss(out[:, 0][mask], data.y[mask])
    low_bound = alpha / 2
    upp_bound = 1 - alpha / 2
    lower = out[:, 1][mask].reshape(-1,1)
    upper = out[:, 2][mask].reshape(-1,1)
    low_loss = torch.mean(torch.max((low_bound - 1) * (label - lower), low_bound * (label - lower)))
    upp_loss = torch.mean(torch.max((upp_bound - 1) * (label - upper), upp_bound * (label - upper)))
    loss = mse_loss/1.6 + low_loss/0.1 + upp_loss/0.05

    if val:
        model2.train()
    else:
        model2.eval()
    mask = data.val_mask if val else data.train_mask
    out2 = model2(data.x, data.edge_index)
    label2 = data.y[mask]-out[mask] #   new labels: X-X_hat
    label3 = upper-lower

    mse_loss2 = F.mse_loss(out2[:,0][mask].reshape(-1,1), label2)
    mse_loss3 = F.mse_loss(out2[:,1][mask].reshape(-1,1), label3)
    loss2 = mse_loss2/11.3  #+ mse_loss3/1
    print('mse_loss of model2')
    print(mse_loss2)
    print('mse_loss of model2')
    print(mse_loss3)

    if val:
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
    if not val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return float(loss),float(loss2)

# DiGAE

In [ ]:
def train_gae_directed(x2, x,edge_index_train2, edge_index_train,edge_weight2, edge_weight, alpha=ALPHA, val=False, edge_index_val=None, sigmoid=False):
    if val:
        model.eval()
    else:
        model.train()
    Z_source, Z_target = model(x, x, edge_tensor, edge_weight_gae_training)
    out_dim = Z_source.shape[-1] // 3
    z_mid_source = Z_source[:, :out_dim]; z_lower_source = Z_source[:, out_dim:2*out_dim]; z_upper_source = Z_source[:, 2*out_dim:]
    z_mid_target = Z_target[:, :out_dim]; z_lower_target = Z_target[:, out_dim:2*out_dim]; z_upper_target = Z_target[:, 2*out_dim:]
    if val:
        out = model.decoder(z_mid_source, z_mid_target, edge_index_val, sigmoid=sigmoid)
        lower = model.decoder(z_lower_source, z_lower_target, edge_index_val, sigmoid=sigmoid)
        upper = model.decoder(z_upper_source, z_upper_target, edge_index_val, sigmoid=sigmoid)
    else:
        out = model.decoder(z_mid_source, z_mid_target, edge_index_train, sigmoid=sigmoid)
        lower = model.decoder(z_lower_source, z_lower_target, edge_index_train, sigmoid=sigmoid)
        upper = model.decoder(z_upper_source, z_upper_target, edge_index_train, sigmoid=sigmoid)

    label = edge_weight
    mse_loss = F.mse_loss(out, label)
    label2 = out - label
    low_bound = alpha / 2; upp_bound = 1 - alpha / 2
    low_loss = torch.mean(torch.max((low_bound - 1) * (label - lower), low_bound * (label - lower)))
    upp_loss = torch.mean(torch.max((upp_bound - 1) * (label - upper), upp_bound * (label - upper)))
    loss = low_loss/1.25 + upp_loss/2.5 + mse_loss/50

    if val:
        model2.train()
    else:
        model2.eval()
    #print('1')

    Z_source2, Z_target2 = model2(x2, x2, edge_tensor2, edge_weight_gae_training2)
    out_dim2 = Z_source2.shape[-1] // 3
    z_mid_source2 = Z_source2[:, :out_dim2]; z_lower_source2 = Z_source2[:, out_dim2:2*out_dim2]; z_upper_source2 = Z_source2[:, 2*out_dim2:]
    z_mid_target2 = Z_target2[:, :out_dim2]; z_lower_target2 = Z_target2[:, out_dim2:2*out_dim2]; z_upper_target2 = Z_target2[:, 2*out_dim2:]
    #print('2')
    if not val:
        out2 = model2.decoder(z_mid_source2, z_mid_target2, edge_index_train, sigmoid=sigmoid)
        #lower2 = model2.decoder(z_lower_source2, z_lower_target2, edge_index_train, sigmoid=sigmoid)
        #upper2 = model2.decoder(z_upper_source2, z_upper_target2, edge_index_train, sigmoid=sigmoid)
    else:
        out2 = model2.decoder(z_mid_source2, z_mid_target2, edge_index_val, sigmoid=sigmoid)
        #lower2 = model2.decoder(z_lower_source2, z_lower_target2, edge_index_val, sigmoid=sigmoid)
        #upper2 = model2.decoder(z_upper_source2, z_upper_target2, edge_index_val, sigmoid=sigmoid)
    #print('3')
    mse_loss2 = F.mse_loss(out2, label2)
    loss2 = mse_loss2/50  #+ mse_loss3/1

    #print('mse_loss of model2')
    #print(mse_loss2)
    if val:
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
    if not val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return float(loss), float(loss2)

In [ ]:
def run_conformal_regression_gae(labels2,labels, lower, upper, alpha=ALPHA, return_prediction_sets=False, return_conditional_coverage=False, score='cqr', x=None, edge_index_calib_test=None):
    num_runs = 100
    if torch.is_tensor(labels):
        labels = labels.detach().numpy()
    if torch.is_tensor(upper):
        upper = upper.detach().numpy()
    if torch.is_tensor(lower):
        lower = lower.detach().numpy()

    n_test_calib = labels.shape[0]
    n_calib = n_test_calib // 2
    idx = np.array([1] * n_calib + [0] * (n_test_calib-n_calib)) > 0

    cov_all = []
    eff_all = []
    if return_conditional_coverage:
        ws_cov_all = []
    if return_prediction_sets:
        pred_set_all = []
        val_labels_all = []
        idx_all = []
    for k in range(num_runs):
        np.random.seed(k)
        np.random.shuffle(idx)
        if return_prediction_sets:
            idx_all.append(idx)
        cal_labels, val_labels = labels[idx], labels[~idx]
        cal_labels2, val_labels2 = labels2[idx], labels2[~idx]
        cal_upper, val_upper = upper[idx], upper[~idx]
        cal_lower, val_lower = lower[idx], lower[~idx]
        if score == 'cqr':
            prediction_sets, cov, eff = cqr(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n_test_calib, alpha)
        elif score == 'qr':
            prediction_sets, cov, eff = qr(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n_test_calib, alpha)
        elif score == 'cqr_new':
            prediction_sets, cov, eff = cqr_new(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n_test_calib, alpha)
        elif score == 'cqr2_new':
            prediction_sets, cov, eff = cqr2_new(cal_labels2, cal_labels, cal_lower, cal_upper,val_labels2, val_labels, val_lower, val_upper, n_test_calib, alpha)

        if return_conditional_coverage:
            ws_cov = worst_slice_coverage(x, edge_index_calib_test, idx, val_labels, prediction_sets)
            if ws_cov is not None:
                ws_cov_all.append(ws_cov)
        cov_all.append(cov)
        eff_all.append(eff)
        if return_prediction_sets:
            pred_set_all.append(prediction_sets)
            val_labels_all.append(val_labels)

    if return_prediction_sets:
        if return_conditional_coverage:
            return cov_all, eff_all, ws_cov_all, pred_set_all, val_labels_all, idx_all
        return cov_all, eff_all, pred_set_all, val_labels_all, idx_all
    else:
        return np.mean(cov_all), np.mean(eff_all)

In [ ]:
def test_gae_directed(best_model, x, train_edge_index, calib_test_edge_index, calib_test_edge_weight, alpha=ALPHA, return_prediction_sets=False, score='cqr', conditional=True, sigmoid=False):
    best_model = best_model.cpu()
    best_model.eval()
    Z_source, Z_target = best_model(x.cpu(), x.cpu(), edge_tensor.cpu(), edge_weight_gae_training.cpu())
    out_dim = Z_source.shape[-1] // 3
    z_mid_source = Z_source[:, :out_dim]; z_lower_source = Z_source[:, out_dim:2*out_dim]; z_upper_source = Z_source[:, 2*out_dim:]
    z_mid_target = Z_target[:, :out_dim]; z_lower_target = Z_target[:, out_dim:2*out_dim]; z_upper_target = Z_target[:, 2*out_dim:]

    out = best_model.decoder(z_mid_source, z_mid_target, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    lower = best_model.decoder(z_lower_source, z_lower_target, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    upper = best_model.decoder(z_upper_source, z_upper_target, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    label2 = calib_test_edge_weight.cpu().detach().numpy() - out.cpu().detach().numpy()
    if conditional:
        return run_conformal_regression_gae(label2, calib_test_edge_weight.cpu(), lower, upper, alpha, return_prediction_sets=return_prediction_sets, return_conditional_coverage=True, x=x, edge_index_calib_test=calib_test_edge_index)

    return run_conformal_regression_gae(label2, calib_test_edge_weight.cpu(), lower, upper, alpha, return_prediction_sets=return_prediction_sets, score=score)

In [ ]:
for i in range(10):
    encoder = DirectedGNN(in_channels=airport.x.shape[-1], hidden_channels=HIDDEN, out_channels=3*OUT, gconv=GNNCONV)
    encoder2 = DirectedGNN(in_channels=airport.x.shape[-1], hidden_channels=HIDDEN, out_channels=1*OUT, gconv=GNNCONV)

    decoder = DirectedInnerProductDecoder()
    decoder2 = DirectedInnerProductDecoder()

    model = GAE(encoder, decoder).to(device)
    model2 =GAE(encoder2, decoder2).to(device)
    x = airport.x.to(device)

    edge_index_train2 = edge_index_train
    edge_weight_train2 = edge_weight_train
    edge_index_train2 = edge_index_train
    edge_weight_train2 =edge_weight_train

    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
    optimizer2 = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


    best_val_loss = float('inf')
    val_loss = float('inf')
    best_model = None
    use_sigmoid = False
    for epoch in range(1, 6*EPOCHS):
        if epoch%4 ==0:
            val1 = True
            val_loss,val_loss2 = train_gae_directed(x,x,edge_index_val2, edge_index_val,edge_weight_val2, edge_weight_val, val=val1, edge_index_val=edge_index_val, sigmoid=use_sigmoid)

        else:
            val1=False
            val_lossn, val_loss2n = train_gae_directed(x, x, edge_index_train2, edge_index_train,edge_weight_train2, edge_weight_train,val=val1, sigmoid=use_sigmoid)


        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)
        #print(f'Epoch: {epoch:03d}, Best validation loss: {val_loss:.4f}')
    _, cov_all, eff_all, pred_set_all, val_labels_all, idx_all = test_gae_directed(best_model.cpu(), x.cpu(), edge_index_train, edge_index_calib_test, edge_weight_calib_test, return_prediction_sets=True, score=SCORE, sigmoid=use_sigmoid, conditional=True)
    print(f"{np.mean(cov_all):.4f}+/-{np.std(cov_all):.4f}, {np.mean(eff_all):.4f}+/-{np.std(eff_all):.4f}")

In [ ]:
#cov_all, eff_all, ws_cov_all, pred_set_all, val_labels_all, idx_all = test_gae_directed(best_model, x, edge_index_train, edge_index_calib_test, edge_weight_calib_test, return_prediction_sets=True, score='cqr_new', sigmoid=use_sigmoid, conditional=True)
cov_all, eff_all, pred_set_all, val_labels_all, idx_all = test_gae_directed(best_model.cpu(), x.cpu(), edge_index_train, edge_index_calib_test, edge_weight_calib_test, return_prediction_sets=True, score=SCORE, sigmoid=use_sigmoid, conditional=False)


In [ ]:
# print(f"{np.mean(cov_all):.4f}+/-{np.std(cov_all):.4f}, {np.mean(eff_all):.4f}+/-{np.std(eff_all):.4f}, {np.mean(ws_cov_all):.4f}+/-{np.std(ws_cov_all):.4f}")
print(f"{np.mean(cov_all):.4f}+/-{np.std(cov_all):.4f}, {np.mean(eff_all):.4f}+/-{np.std(eff_all):.4f}")

0.9504+/-0.0167, 3.7640+/-0.0497


# GAE

In [ ]:
def train_gae(x, edge_index_train, edge_weight, alpha=ALPHA, val=False, edge_index_val=None, sigmoid=False):
    if val:
        model.eval()
    else:
        model.train()
    Z = model(x, edge_tensor, edge_weight_gae_training)
    out_dim = Z.shape[-1] // 3
    z_mid = Z[:, :out_dim]; z_lower = Z[:, out_dim:2*out_dim]; z_upper = Z[:, 2*out_dim:]
    if val:
        out = model.decoder(z_mid, edge_index_val, sigmoid=sigmoid)
        lower = model.decoder(z_lower, edge_index_val, sigmoid=sigmoid)
        upper = model.decoder(z_upper, edge_index_val, sigmoid=sigmoid)
    else:
        out = model.decoder(z_mid, edge_index_train, sigmoid=sigmoid)
        lower = model.decoder(z_lower, edge_index_train, sigmoid=sigmoid)
        upper = model.decoder(z_upper, edge_index_train, sigmoid=sigmoid)

    label = edge_weight
    mse_loss = F.mse_loss(out, label)
    low_bound = alpha / 2; upp_bound = 1 - alpha / 2
    low_loss = torch.mean(torch.max((low_bound - 1) * (label - lower), low_bound * (label - lower)))
    upp_loss = torch.mean(torch.max((upp_bound - 1) * (label - upper), upp_bound * (label - upper)))
    loss = low_loss + upp_loss # mse_loss +

    if not val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return float(loss)

In [ ]:
def run_conformal_regression_gae2(labels2,labels, lower, upper, alpha=ALPHA, return_prediction_sets=False, return_conditional_coverage=False, score='cqr', x=None, edge_index_calib_test=None):
    num_runs = 100
    if torch.is_tensor(labels):
        labels = labels.detach().numpy()
    if torch.is_tensor(upper):
        upper = upper.detach().numpy()
    if torch.is_tensor(lower):
        lower = lower.detach().numpy()

    n_test_calib = labels.shape[0]
    n_calib = n_test_calib // 2
    idx = np.array([1] * n_calib + [0] * (n_test_calib-n_calib)) > 0

    cov_all = []
    eff_all = []
    if return_conditional_coverage:
        ws_cov_all = []
    if return_prediction_sets:
        pred_set_all = []
        val_labels_all = []
        idx_all = []
    for k in range(num_runs):
        np.random.seed(k)
        np.random.shuffle(idx)
        if return_prediction_sets:
            idx_all.append(idx)
        cal_labels, val_labels = labels[idx], labels[~idx]
        cal_labels2, val_labels2 = labels2[idx], labels2[~idx]
        cal_upper, val_upper = upper[idx], upper[~idx]
        cal_lower, val_lower = lower[idx], lower[~idx]
        if score == 'cqr':
            prediction_sets, cov, eff = cqr(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n_test_calib, alpha)
        elif score == 'qr':
            prediction_sets, cov, eff = qr(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n_test_calib, alpha)
        elif score == 'cqr_new':
            prediction_sets, cov, eff = cqr_new(cal_labels, cal_lower, cal_upper, val_labels, val_lower, val_upper, n_test_calib, alpha)
        elif score == 'cqr2_new':
            prediction_sets, cov, eff = cqr2_new(cal_labels2, cal_labels, cal_lower, cal_upper,val_labels2, val_labels, val_lower, val_upper, n_test_calib, alpha)

        if return_conditional_coverage:
            ws_cov = worst_slice_coverage(x, edge_index_calib_test, idx, val_labels, prediction_sets)
            if ws_cov is not None:
                ws_cov_all.append(ws_cov)
        cov_all.append(cov)
        eff_all.append(eff)
        if return_prediction_sets:
            pred_set_all.append(prediction_sets)
            val_labels_all.append(val_labels)

    if return_prediction_sets:
        if return_conditional_coverage:
            return cov_all, eff_all, ws_cov_all, pred_set_all, val_labels_all, idx_all
        return cov_all, eff_all, pred_set_all, val_labels_all, idx_all
    else:
        return np.mean(cov_all), np.mean(eff_all)

In [ ]:
def train_gae2o(x2, x, edge_index_train2, edge_index_train, edge_weight2, edge_weight, alpha=ALPHA, val=False, edge_index_val=None, sigmoid=False):
    if val:
        model.eval()
    else:
        model.train()
    Z = model(x, edge_tensor, edge_weight_gae_training)
    out_dim = Z.shape[-1] // 3
    z_mid = Z[:, :out_dim]; z_lower = Z[:, out_dim:2*out_dim]; z_upper = Z[:, 2*out_dim:]
    if val:
        out = model.decoder(z_mid, edge_index_val, sigmoid=sigmoid)
        lower = model.decoder(z_lower, edge_index_val, sigmoid=sigmoid)
        upper = model.decoder(z_upper, edge_index_val, sigmoid=sigmoid)
    else:
        out = model.decoder(z_mid, edge_index_train, sigmoid=sigmoid)
        lower = model.decoder(z_lower, edge_index_train, sigmoid=sigmoid)
        upper = model.decoder(z_upper, edge_index_train, sigmoid=sigmoid)

    label = edge_weight
    mse_loss = F.mse_loss(out, label)
    label2 = out - label
    low_bound = alpha / 2; upp_bound = 1 - alpha / 2
    low_loss = torch.mean(torch.max((low_bound - 1) * (label - lower), low_bound * (label - lower)))
    upp_loss = torch.mean(torch.max((upp_bound - 1) * (label - upper), upp_bound * (label - upper)))
    loss = low_loss + upp_loss # mse_loss +


    if val:
        model2.train()
    else:
        model2.eval()

    Z2 = model2(x2, edge_tensor2, edge_weight_gae_training2)
    out_dim2 = Z2.shape[-1] // 3
    z_mid2 = Z2[:, :out_dim2]; z_lower2 = Z2[:, out_dim2:2*out_dim2]; z_upper2 = Z2[:, 2*out_dim2:]
    if not val:
        out2 = model2.decoder(z_mid2, edge_index_val2, sigmoid=sigmoid)
        #lower = model.decoder(z_lower, edge_index_val, sigmoid=sigmoid)
        #upper = model.decoder(z_upper, edge_index_val, sigmoid=sigmoid)
    else:
        out2 = model2.decoder(z_mid2, edge_index_train2, sigmoid=sigmoid)
        #lower = model.decoder(z_lower, edge_index_train, sigmoid=sigmoid)
        #upper = model.decoder(z_upper, edge_index_train, sigmoid=sigmoid)

    #mse_loss2 = F.mse_loss(out2, label2)

    #print(label2)
    #print('outttttttttttttttttttttt2')
    #print(out2)
    mse_loss2 = F.mse_loss(label2[0:215], out2)

    loss2 =  mse_loss2


    if not val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if val:
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()

    return float(loss), float(loss2)

In [ ]:
def train_gae2(x2, x, edge_index_train2, edge_index_train, edge_weight2, edge_weight, alpha=ALPHA, val=False, edge_index_val=None, sigmoid=False):


    if val:
        model.eval()
    else:
        model.train()
    Z_source = model( x, edge_tensor, edge_weight_gae_training)
    out_dim = Z_source.shape[-1] // 3
    z_mid_source = Z_source[:, :out_dim]; z_lower_source = Z_source[:, out_dim:2*out_dim]; z_upper_source = Z_source[:, 2*out_dim:]
    if val:
        out = model.decoder(z_mid_source, edge_index_val, sigmoid=sigmoid)
        lower = model.decoder(z_lower_source, edge_index_val, sigmoid=sigmoid)
        upper = model.decoder(z_upper_source, edge_index_val, sigmoid=sigmoid)
    else:
        out = model.decoder(z_mid_source, edge_index_train, sigmoid=sigmoid)
        lower = model.decoder(z_lower_source, edge_index_train, sigmoid=sigmoid)
        upper = model.decoder(z_upper_source, edge_index_train, sigmoid=sigmoid)

    label = edge_weight
    mse_loss = F.mse_loss(out, label)
    label2 = out - label
    low_bound = alpha / 2; upp_bound = 1 - alpha / 2
    low_loss = torch.mean(torch.max((low_bound - 1) * (label - lower), low_bound * (label - lower)))
    upp_loss = torch.mean(torch.max((upp_bound - 1) * (label - upper), upp_bound * (label - upper)))
    loss = low_loss/1.25 + upp_loss/2.5 + mse_loss/50

    #return float(loss)
    #print('upper losss')
    #print(upp_loss)
    #print('lower loss')
    #print(low_loss)
    if val:
        model2.train()
    else:
        model2.eval()
    #print('1')

    Z_source2 = model2( x2, edge_tensor2, edge_weight_gae_training2)
    out_dim2 = Z_source2.shape[-1] // 3
    z_mid_source2 = Z_source2[:, :out_dim2]; z_lower_source2 = Z_source2[:, out_dim2:2*out_dim2]; z_upper_source2 = Z_source2[:, 2*out_dim2:]
    #print('2')
    if not val:
        out2 = model2.decoder(z_mid_source2, edge_index_train, sigmoid=sigmoid)
        #lower2 = model2.decoder(z_lower_source2, z_lower_target2, edge_index_train, sigmoid=sigmoid)
        #upper2 = model2.decoder(z_upper_source2, z_upper_target2, edge_index_train, sigmoid=sigmoid)
    else:
        out2 = model2.decoder(z_mid_source2, edge_index_val, sigmoid=sigmoid)
        #lower2 = model2.decoder(z_lower_source2, z_lower_target2, edge_index_val, sigmoid=sigmoid)
        #upper2 = model2.decoder(z_upper_source2, z_upper_target2, edge_index_val, sigmoid=sigmoid)
    #print('3')
    # 调整 out2 尺寸与 label2 相同
    #out2_resized = F.interpolate(out2, size=(label2.size()), mode='bilinear', align_corners=False)

    # 计算 MSE Loss
    #mse_loss2 = F.mse_loss(out2_resized, label2)
    mse_loss2 = F.mse_loss(out2, label2)
    loss2 = mse_loss2/50  #+ mse_loss3/1

    #print('mse_loss of model2')
    #print(mse_loss2)
    if val:
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
    if not val:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return float(loss), float(loss2)

In [ ]:
def test_gae(best_model, x, train_edge_index, calib_test_edge_index, calib_test_edge_weight, alpha=ALPHA, return_prediction_sets=False, score='cqr', conditional=True, sigmoid=False):
    best_model = best_model.cpu()
    best_model.eval()
    Z = best_model(x.cpu(), edge_tensor.cpu(), edge_weight_gae_training.cpu())
    out_dim = Z.shape[-1] // 3
    z_mid = Z[:, :out_dim]; z_lower = Z[:, out_dim:2*out_dim]; z_upper = Z[:, 2*out_dim:]
    out = model.decoder(z_mid, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    lower = model.decoder(z_lower, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    upper = model.decoder(z_upper, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    if conditional:
        return run_conformal_regression_gae(calib_test_edge_weight.cpu(), lower, upper, alpha, return_prediction_sets=return_prediction_sets, return_conditional_coverage=True, x=x, edge_index_calib_test=calib_test_edge_index)

    return run_conformal_regression_gae(calib_test_edge_weight.cpu(), lower, upper, alpha, return_prediction_sets=return_prediction_sets, score=score)

In [ ]:
def test_gae2(best_model, x, train_edge_index, calib_test_edge_index, calib_test_edge_weight, alpha=ALPHA, return_prediction_sets=False, score='cqr', conditional=True, sigmoid=False):
    best_model = best_model.cpu()
    best_model.eval()
    Z = best_model(x.cpu(), edge_tensor.cpu(), edge_weight_gae_training.cpu())
    out_dim = Z.shape[-1] // 3
    z_mid = Z[:, :out_dim]; z_lower = Z[:, out_dim:2*out_dim]; z_upper = Z[:, 2*out_dim:]
    out = model.decoder(z_mid, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    lower = model.decoder(z_lower, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    upper = model.decoder(z_upper, calib_test_edge_index.cpu(), sigmoid=sigmoid)
    label2 = calib_test_edge_weight.cpu().detach().numpy() - out.cpu().detach().numpy()


    if conditional:
        return run_conformal_regression_gae2(label2,calib_test_edge_weight.cpu(), lower.detach().cpu().numpy(), upper.detach().cpu().numpy(), alpha, return_prediction_sets=return_prediction_sets, return_conditional_coverage=True, x=x, edge_index_calib_test=calib_test_edge_index)

    return run_conformal_regression_gae2(label2, calib_test_edge_weight.cpu(), lower.detach().cpu().numpy(), upper.detach().cpu().numpy(), alpha, return_prediction_sets=return_prediction_sets, score=score)

In [ ]:
for i in range(10):
    encoder = GNN(in_channels=airport.x.shape[-1], hidden_channels=HIDDEN, out_channels=3*OUT, gconv=GNNCONV)
    decoder = InnerProductDecoder()
    model = GAE(encoder, decoder).to(device)

    encoder2 = GNN(in_channels=airport.x.shape[-1], hidden_channels=HIDDEN, out_channels=3*OUT, gconv=GNNCONV)
    decoder2 = InnerProductDecoder()
    model2 = GAE(encoder2, decoder2).to(device)
    #print(model2)
    x = airport.x.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=LR, weight_decay=WD)

    edge_index_train2 = edge_index_train
    edge_weight_train2 = edge_weight_train
    edge_index_train2 = edge_index_train
    edge_weight_train2 =edge_weight_train

    best_val_loss = float('inf')
    val_loss = float('inf')
    best_model = None
    use_sigmoid = False

    for epoch in range(1, 6*EPOCHS):
        if epoch%4 ==0:
            val1 = True
            val_loss,val_loss2 = train_gae2(x, x, edge_index_val2, edge_index_val, edge_weight_val2, edge_weight_val,val=val1, edge_index_val=edge_index_val,sigmoid=use_sigmoid)

        else:
            val1=False
            val_lossn, val_loss2n = train_gae2(x, x,edge_index_train2, edge_index_train,edge_weight_train2, edge_weight_train,val=val1, sigmoid=use_sigmoid)

        #loss = train_gae(x, edge_index_train, edge_weight_train, sigmoid=use_sigmoid)
        #if epoch % 1 == 1:
            #print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        #val_loss = train_gae(x, edge_index_train, edge_weight_val, val=True, edge_index_val=edge_index_val, sigmoid=use_sigmoid)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)
            #print(f'Epoch: {epoch:03d}, Best validation loss: {val_loss:.4f}')
    _,cov_all, eff_all, pred_set_all, val_labels_all, idx_all = test_gae2(best_model, x, edge_index_train, edge_index_calib_test, edge_weight_calib_test, return_prediction_sets=True, score=SCORE, sigmoid=use_sigmoid, conditional=True)
    print(f"{np.mean(cov_all):.4f}+/-{np.std(cov_all):.4f}, {np.mean(eff_all):.4f}+/-{np.std(eff_all):.4f}")

In [ ]:
cov_all, eff_all, pred_set_all, val_labels_all, idx_all = test_gae2(best_model, x, edge_index_train, edge_index_calib_test, edge_weight_calib_test, return_prediction_sets=True, score=SCORE, sigmoid=use_sigmoid, conditional=False)
print(f"{np.mean(cov_all):.4f}+/-{np.std(cov_all):.4f}, {np.mean(eff_all):.4f}+/-{np.std(eff_all):.4f}")

0.9520+/-0.0159, 3.4561+/-0.0453


# Line Graph